## TIC TAC TOE


This project is based upon Reinforcement Learning which is a technique in which most of the bots and AI agent learns to do things.

In this project i will make a simple AI bot with whom i can play a game of Tic Tac Toe. 


And it the bot will learn to play the game from the mistakes it would make in the process of doing so.

# Importing the important libraries and setting initial parameters

In [2]:
import random
import json

# gamma is the discount factor
gamma = 0.9 

# alpha is the learning rate
alpha = 0.1

A function to find who won the game or has the match tie

In [3]:
def winner(state):
    for i in range(3):
        s = state[i][0] + state[i][1] + state[i][2]
        if (s==0):
            return 0
        elif (s==3):
            return 1
        
        s1 = state[0][i] + state[1][i] + state[2][i]
        if (s1==0):
            return 0
        elif (s1==3):
            return 1
        
    ld = state[0][0] + state[1][1] + state[2][2]
    rd = state[0][2] + state[1][1] + state[2][0]
    
    if(ld==0):
        return 0
    elif(ld==3):
        return 1
    
    if(rd==0):
        return 0
    elif(rd==3):
        return 1
    
    return -1


A function to check whether game has ended or not

In [4]:
def game_end(state):
    
    k = 0
    for i in range(3):
        for j in range(3):
            if(state[i][j] == -10):
                k += 1

    if(k==0):
        return True
    else:
        for i in range(3):
            s = state[i][0] + state[i][1] + state[i][2]
            if (s==0 or s==3):
                return True
            s1 = state[0][i] + state[1][i] + state[2][i]
            if (s1==0 or s1==3):
                return True
        ld = state[0][0] + state[1][1] + state[2][2]
        rd = state[0][2] + state[1][1] + state[2][0]
        
        if(ld==0 or ld==3):
            return True
        if(rd==0 or rd==3):
            return True
        return False


A function to display the current state of the game after each move

In [5]:
def display(grid):
    for i in range(3):
        for j in range(3):
            if(grid[i][j]==1):
                print("X", end = " ")
            elif(grid[i][j]==0):
                print("O", end = " ")
            else:
                print("_", end = " ")
        print()


An important function to check if the current state of the game is previously visited and is present in the value dictionary

In [6]:
# grid is the game state
# value is the dictionary mapping the game state to a value
def is_present(grid, value):
    
    G0 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    G1 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    G2 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    G3 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    G4 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    G5 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    G6 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    G7 = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    

    # In each step here json is used to convert the gris state which 
    # is a 2d matrix to a mappable feature that could be mapped to a value
    G0 = json.dumps(list(map(list, grid)))
    
    for i in range(3):
        for j in range(3):
            G1[i][j] = grid[i][2-j]
    G1 = json.dumps(list(map(list, G1)))

    for i in range(3):
        for j in range(3):
            G2[i][j] = grid[2-i][j]
    G2 = json.dumps(list(map(list, G2)))
    
    for i in range(3):
        for j in range(3):
            G3[i][j] = grid[2-i][2-j]
    G3 = json.dumps(list(map(list, G3)))
    
    for i in range(3):
        for j in range(3):
            G4[i][j] = grid[j][i]
    G4 = json.dumps(list(map(list, G4)))
    
    for i in range(3):
        for j in range(3):
            G5[i][j] = grid[j][2-i]
    G5 = json.dumps(list(map(list, G5)))
    
    for i in range(3):
        for j in range(3):
            G6[i][j] = grid[2-j][i]
    G6 = json.dumps(list(map(list, G6)))

    for i in range(3):
        for j in range(3):
            G7[i][j] = grid[2-j][2-i]
    G7 = json.dumps(list(map(list, G7)))
    
    if(G0 in value):
        return (value[G0])
    elif(G1 in value):
        return (value[G1])
    elif(G2 in value):
        return (value[G2])
    elif(G3 in value):
        return (value[G3])
    elif(G4 in value):
        return (value[G4])
    elif(G5 in value):
        return (value[G5])
    elif(G6 in value):
        return (value[G6])
    elif(G7 in value):
        return (value[G7])
    else:
        value[G0] = 0
        return 0


This is a very important function that makes the agent intelligent fast
and makes it learn the game more robustly.
In this function there is an opponent agent with preprogrammed strategies as follows:

1: if the AI agent is going to win with his 2 marks present in one line, then it would block the last mark on the line.

2: If this agent has 2 marks on one line , its policy will be to always do a mark on the last position in the line and thus win the game.


I will train my AI agent against this preprogrammed agent and make it learn the game more robustly.

In [7]:
def opponent_policy(state, action):
    
    for i in range(3):
        p = state[i][0] + state[i][1] + state[i][2]
        if(p==-8):
            for j in range(3):
                if(state[i][j]==-10):
                    return ([i,j])
        p1 = state[0][i] + state[1][i] + state[2][i]
        if(p1==-8):
            for j in range(3):
                if(state[j][i]==-10):
                    return ([j,i])
        
    ld1 = state[0][0] + state[1][1] + state[2][2]
    rd1 = state[0][2] + state[1][1] + state[2][0]
    
    if(ld1==-8):
        for i in range(3):
            if(state[i][i]==-10):
                return ([i,i])
    
    if(rd1==-8):
        for i in range(3):
            if(state[i][2-i]==-10):
                return ([i,2-i])
    
    for i in range(3):
        p = state[i][0] + state[i][1] + state[i][2]
        if(p==-10):
            for j in range(3):
                if(state[i][j]==-10):
                    return ([i,j])
        p1 = state[0][i] + state[1][i] + state[2][i]
        if(p1==-10):
            for j in range(3):
                if(state[j][i]==-10):
                    return ([j,i])
        
    ld1 = state[0][0] + state[1][1] + state[2][2]
    rd1 = state[0][2] + state[1][1] + state[2][0]
    
    if(ld1==-10):
        for i in range(3):
            if(state[i][i]==-10):
                return ([i,i])
    
    if(rd1==-10):
        for i in range(3):
            if(state[i][2-i]==-10):
                return ([i,2-i])
    
    return (random.choice(action))
 

A funtion to play the episodes of the game in which the AI agent plays against the random choice agent , which is not a good player.

In [8]:
def play_episode(s, episode):

    # action is a list of actions available for the agent to perform
    # s is the action performed
    action = []
    grid = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]] # this is the empty grid state
    for i in range(3):
        for j in range(3):
            action.append([i, j])
           
    action.remove(s)
    grid[s[0]][s[1]] = 1
    temp1 = list(map(list, grid))
    episode.append(temp1)

    # play the episode until someone wins or match ties
    while(not(game_end(grid))):
        # agent performs a random move
        agent = random.choice(action)
        action.remove(agent)
        grid[agent[0]][agent[1]] = 0
        temp = list(map(list, grid))
        episode.append(temp)
        
        if((game_end(grid))):
            break
        # random move agent also performs a random move
        user = random.choice(action)
        action.remove(user)
        grid[user[0]][user[1]] = 1
        temp = list(map(list, grid))
        episode.append(temp)
        
        if((game_end(grid))):
            break
    

Initializing the parameters

In [9]:
value = {}
h = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
h1 = json.dumps(h)
value[h1] = 0

Training the AI agent against the random policy agent for 10000 episodes

In [10]:
for ep in  range(10000):
    # I have kept the intial action only 4 because rest of the actions are
    # symmetrical with these actions
    in_act = [[0,0],[0,1],[1,0],[1,1]]

    for ch in range(4):
        
        s = in_act[ch]
        # episode is a list which will consist of all the game states in a complete episode of the game
        episode = []
        play_episode(s, episode) 
        end = episode[len(episode)-1]
        end_hash = json.dumps(end)
        if(winner(end)==0):
          # winning state has a large positive value of 5
            value[end_hash] = 5
        elif(winner(end)==1):
          # losing state has a large negative value of 5
            value[end_hash] = -5
        else:
          # match tie state has a intermediatevalue of positive 2
            value[end_hash] = 2
          

        # performing the TD learning (temporal difference learning) on each episode
        for i in range(len(episode)-1):
            s = is_present(episode[i], value)
            ns = is_present(episode[i+1], value)
            S = json.dumps(episode[i])
            if(i==len(episode)-2 and value[json.dumps(episode[i+1])]==5):
                value[S] = s + alpha*(2 + gamma*ns - s)
            elif(i==len(episode)-2 and value[json.dumps(episode[i+1])]==-5):
                value[S] = s + alpha*(-2 + gamma*ns - s)
            elif(i==len(episode)-2 and value[json.dumps(episode[i+1])]==2):
                value[S] = s + alpha*(1 + gamma*ns - s)
            else:
                value[S] = s + alpha*(-1 + gamma*ns - s)
  

Making a copy of the values of the states the agent could find

In [11]:
value1 = value.copy()

A function to make the AI agent play gainst the preprogrammed agent

In [12]:
def play_intelligent_episode(s,episode):
    
    action = []
    grid = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    for i in range(3):
        for j in range(3):
            action.append([i, j])
           
    action.remove(s)
    grid[s[0]][s[1]] = 1
    temp1 = list(map(list, grid))
    episode.append(temp1)
    
    while(not(game_end(grid))):
        
        Max = -100.0
        
        # In this loop the agent always performs the best move it could make
        for act in action:
            Grid = list(map(list, grid))
            Grid[act[0]][act[1]] = 0
            val = is_present(Grid, value1)
            Max = max(Max, val)
            if(Max == val):
                agent = act
                
        action.remove(agent)
        grid[agent[0]][agent[1]] = 0
        temp = list(map(list, grid))
        episode.append(temp)
        
        
        
        if((game_end(grid))):
            break
        
        # user performs with the preprogrammed agent policy
        user = opponent_policy(grid, action)
        action.remove(user)
        grid[user[0]][user[1]] = 1
        temp = list(map(list, grid))
        episode.append(temp)
        
        
        
        if((game_end(grid))):
            break
        

Setting the learning rate to a higher value so that it could learn more robustly

In [13]:
alpha = 0.18

Training the AI agent against preprogrammed agent for 1000 episodes

In [14]:
for ep in  range(1000):
  
    in_act = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]
    episode = []
    S = random.choice(in_act)

    # playing with the preprogrammed agent
    play_intelligent_episode(S, episode) 
    end = episode[len(episode)-1]
    end_hash = json.dumps(end)
    if(winner(end)==0):
        value1[end_hash] = 5
    elif(winner(end)==1):
        value1[end_hash] = -5
    else:
        value1[end_hash] = 2

    # performing the TD learning step
    for i in range(len(episode)-1):
        s = is_present(episode[i], value1)
        ns = is_present(episode[i+1], value1)
        S = json.dumps(episode[i])
        if(i==len(episode)-2 and value1[json.dumps(episode[i+1])]==5):
            value1[S] = s + alpha*(2 + gamma*ns - s)
        elif(i==len(episode)-2 and value1[json.dumps(episode[i+1])]==-5):
            value1[S] = s + alpha*(-2 + gamma*ns - s)
        elif(i==len(episode)-2 and value1[json.dumps(episode[i+1])]==2):
            value1[S] = s + alpha*(1 + gamma*ns - s)
        else:
            value1[S] = s + alpha*(-1 + gamma*ns - s)
   

A function so that we could play with the agent after it gets trained

In [15]:
def play_userFirst(s, value, episode):
    
    action = []
    grid = [[-10, -10, -10], [-10, -10, -10], [-10, -10, -10]]
    for i in range(3):
        for j in range(3):
            action.append([i, j])
           
    action.remove(s)
    grid[s[0]][s[1]] = 1
    temp1 = list(map(list, grid))
    episode.append(temp1)
    print("your move :-")
    display(grid)
    print()
    while(not(game_end(grid))):
        
        Max = -100.0
        
        for act in action:
            Grid = list(map(list, grid))
            Grid[act[0]][act[1]] = 0
            val = is_present(Grid, value)
            Max = max(Max, val)
            if(Max == val):
                agent = act
                
        action.remove(agent)
        grid[agent[0]][agent[1]] = 0
        temp = list(map(list, grid))
        episode.append(temp)
        
        print("agent's move :-")
        display(grid)
        print()
        
        if((game_end(grid))):
            break
        
        print("enter your next move")
        user = [int(x) for x in input().split(",")]
        action.remove(user)
        grid[user[0]][user[1]] = 1
        temp = list(map(list, grid))
        episode.append(temp)
        
        print("your move :-")
        display(grid)
        print()
        
        if((game_end(grid))):
            break
        
    if(winner(grid)==1):
        print("You Win")
    elif(winner(grid)==0):
        print("I Win")
    else:
        print("Mtch Tie")


Two important functions such that while playing with me the agent can learn from us too

In [17]:
def start_user1():
    alp = 0.3
    print("start the game")
    print("format for input: ")
    print("position for your mark: n,n")
    print("for example:")
    print("1,1")
    print("above input is for center mark")
    
    S = [int(x) for x in input().split(",")]
    
    episode = []
    play_userFirst(S, value1, episode)
    end = episode[len(episode)-1]
    end_hash = json.dumps(end)
    if(winner(end)==0):
        value1[end_hash] = 5
    elif(winner(end)==1):
        value1[end_hash] = -5
    else:
        value1[end_hash] = 2
    for i in range(len(episode)-1):
        s = is_present(episode[i], value1)
        ns = is_present(episode[i+1], value1)
        S = json.dumps(episode[i])
        if(i==len(episode)-2 and value1[json.dumps(episode[i+1])]==5):
            value1[S] = s + alp*(2 + gamma*ns - s)
        elif(i==len(episode)-2 and value1[json.dumps(episode[i+1])]==-5):
            value1[S] = s + alp*(-2 + gamma*ns - s)
        elif(i==len(episode)-2 and value1[json.dumps(episode[i+1])]==2):
            value1[S] = s + alp*(1 + gamma*ns - s)
        else:
            value1[S] = s + alp*(-1 + gamma*ns - s)

       



def start_game():
        
    t1 = 1
    while(t1 is not 0):
        start_user1()
        print("do u want to play more 1 for yes 0 for no")
        t1 = int(input())
        if(t1==0):
          print("It was nice to play with you. I learned few more things from you")


# Start the Game

In [19]:
start_game()

start the game
1,1
your move :-
_ _ _ 
_ X _ 
_ _ _ 

agent's move :-
_ O _ 
_ X _ 
_ _ _ 

enter your next move
0,0
your move :-
X O _ 
_ X _ 
_ _ _ 

agent's move :-
X O _ 
_ X _ 
_ _ O 

enter your next move
2,0
your move :-
X O _ 
_ X _ 
X _ O 

agent's move :-
X O _ 
_ X O 
X _ O 

enter your next move
1,0
your move :-
X O _ 
X X O 
X _ O 

You Win
do u want to play more 1 for yes 0 fro no
1
start the game
1,1
your move :-
_ _ _ 
_ X _ 
_ _ _ 

agent's move :-
_ _ O 
_ X _ 
_ _ _ 

enter your next move
2,2
your move :-
_ _ O 
_ X _ 
_ _ X 

agent's move :-
O _ O 
_ X _ 
_ _ X 

enter your next move
0,1
your move :-
O X O 
_ X _ 
_ _ X 

agent's move :-
O X O 
_ X _ 
_ O X 

enter your next move
1,2
your move :-
O X O 
_ X X 
_ O X 

agent's move :-
O X O 
O X X 
_ O X 

enter your next move
2,0
your move :-
O X O 
O X X 
X O X 

Mtch Tie
do u want to play more 1 for yes 0 fro no
1
start the game
1,1
your move :-
_ _ _ 
_ X _ 
_ _ _ 

agent's move :-
_ _ O 
_ X _ 
_ _ _ 

enter you